In [1]:
import tools
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# import data 
wd = Path.cwd()
data = pd.read_csv(wd.parent/'data'/'data_transformed'/'outage_bunch_est.csv', parse_dates=['start_date', 'end_date'])


C:\Users\paula\anaconda3\envs\dehli\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# exchange rate Rupies to USD
rp_usd = 0.014

# define parameters 
phi = 50/60 
phi120 = 100/60


$K(\sigma,\pi, \phi) = (\pi + \phi)^{\alpha / (1+\alpha)} \sigma (1 + 1/\alpha)$

Calculating counterfactuals: now that we have sigma, we can compute any counterfactual cost we want, but choosing for example different parameter values for $\pi$ or $\phi$. The steps are:

1- given the sigmas and our chosen parameter values, calculation the implied xs (from xopt)

2- then given xopt and the parameters chosen we calculate K.

In [6]:
# calculate K 
data = data[(data.alpha != 0) & (data.pi!= 0) ]

# method 1: use function tools.K
#counterfactual cost without a fine (phi=0)
data['cost_cf'] = data.apply(lambda row: tools.K(row.duration_cf, sigma = row.sigma, pi = row.pi, phi=0, phi120=0, alpha = row.alpha, N = row.noofcustomersaffected), axis = 1)
# what are the right values for phi?
data['cost_real'] = data.apply(lambda row: tools.K(row.duration_minutes,sigma = row.sigma, pi = row.pi, phi=phi, phi120=phi120, alpha = row.alpha, N = row.noofcustomersaffected), axis = 1)

#this is wrong
# take xopt and double pi
# phi = 0 ??
data['xopt'] = data.apply(lambda row: tools.xopt(row.sigma, row.pi, 0, row.alpha, row.noofcustomersaffected), axis = 1)
data['cost_2pi'] = data.apply(lambda row: tools.K(row.xopt, sigma = row.sigma, pi = 2*row.pi, phi=0, phi120=0, alpha = row.alpha, N = row.noofcustomersaffected), axis = 1)

# get difference
# why do we multiply ba 0.014?
print((data.cost_real.sum() - data.cost_cf.sum())*rp_usd) # 0.014 Rupies to USD
print('doubling pi delta PS:',(data.cost_real.sum() -data.cost_2pi.sum())*rp_usd)

C:\Users\paula\anaconda3\envs\dehli\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


64890808.408024155
doubling pi delta PS: (-8.163495391419669e+38-1.0880652599377983e-78j)


In [ ]:
#print('doubling pi delta PS:',(data.cost_cf_2pi.sum() -data.cost_cf.sum())*0.014)

# Utility of consumer  𝑖  experiencing an outage:

$U_i=\delta T−\pi (T-x)−(p' + \chi)x + FINE $

T is total time (e.g. 24 hours)

Change in consumer surplus:

$ \Delta U_i = (x^{data} - x^{counterfactual})*(\pi-p')$

Since $x^{data} < x^{counterfactual}$ and $𝑝′≥\pi$, and $\chi>0$, $\Delta U_i\geq0 $


Assumptions:

1-  

2- pollution during an outage is only local

Utility of consumer not experiencing an outage:
$U_i = \delta T - p T - \chi \sum_{i} x_{-i} $

In [7]:
# calculate fine for each outage
data['fine_real'] = data.apply(lambda row: tools.V(phi, phi120, row.duration_minutes, N = row.noofcustomersaffected), axis=1)

data['fine_cf'] = data.apply(lambda row: tools.V(0, 0, row.duration_cf, N = row.noofcustomersaffected), axis=1)

In [8]:
# total time
start = data.start_date.min()
end = data.end_date.max()

time_diff = end-start
T = time_diff.total_seconds() / 60
print('total time in minutes:',T)
#T = 12*365

# health cost
#chi = 391000000/data.duration_minutes.sum()
chi = 515
print('chi:', chi)

# at outage level
data['pprime'] = data.apply(lambda row: (row.unservedmuduetooutage * 1000000/ row.duration_minutes ) * 21 , axis=1) # is that right?
print('median pprime:', data.pprime.median())

### change in CS: xbunch and xcounter

data['CS_cf'] = data.apply(lambda row: (- row.pi * (T-row.duration_cf) - row.pprime*row.duration_cf), axis = 1)
##data['CS_cf_nopol'] = data.apply(lambda row: (- pi * (T-row.duration_cf) - pprime*row.duration_cf ), axis = 1)
#data['CS_cf_2pi'] = data.apply(lambda row: (- 2*pi * (T-row.duration_cf) - pprime*row.duration_cf - chi*row.duration_cf, axis = 1)
#data['CS_cf_nopol_2pi'] = data.apply(lambda row: (- 2*pi * (T-row.duration_cf) - pprime*row.duration_cf ), axis = 1)

data['CS_real'] = data.apply(lambda row: (- row.pi * (T-row.duration_minutes) - row.pprime*row.duration_minutes + row.fine_real), axis = 1)
#data['CS_real_nopol'] = data.apply(lambda row: (- pi * (T-row.duration_minutes) - pprime*row.duration_minutes + row.fine_real), axis = 1)


# get difference
print('CS with pollution', (data.CS_real.sum() - data.CS_cf.sum())*rp_usd) #, 'CS No Pollution', (data.CS_real_nopol.sum() - data.CS_cf_nopol.sum())*0.014,)
#print('CS double PI', (data.CS_cf_2pi.sum() - data.CS_cf.sum())*0.014)

total time in minutes: 1137600.0
chi: 515
median pprime: 122.39999999999999
CS with pollution 65011416.01119922


In [9]:
#back of the envelope calculation (total kwh lost * price difference per kwh)--> corresponds to counterfactual of removing outages completely
print(data.unservedmuduetooutage.sum()*10000000* (7-18))

-6413360069.489999


In [10]:
## Total change in Welfare 35357207.35

print('change in welfare with pollution is: USD', (data.CS_real.sum() - data.CS_cf.sum() - (data.cost_real.sum() - data.cost_cf.sum()))*rp_usd) #, 'change in welfare without pollution is: USD', (data.CS_real_nopol.sum() - data.CS_cf_nopol.sum() - (data.cost_real.sum() - data.cost_cf.sum()))*0.014)

change in welfare with pollution is: USD 120607.60317506218


In [11]:
print((data.duration_minutes.sum() - data.duration_cf.sum())/91230) # were does this number come from?

-0.45609868829697287
